In [1]:
import numpy as np
import pandas as pd
import logging
import os
import glob
import warnings
import regex as re
import torch
import argparse
import random
import itertools
import random
import ast
import sys
import ast

from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import simpletransformers
from simpletransformers.t5 import T5Model, T5Args
from datasets import load_dataset, get_dataset_split_names
from transformers import AdamW

os.environ['TORCH_HOME'] = '/scratch/wadhwa.s/cache/'
os.environ['HF_HOME'] = '/scratch/wadhwa.s/cache'
os.environ['TRANSFORMERS_CACHE'] = '/scratch/wadhwa.s/cache'


warnings.filterwarnings("ignore")

/home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ade = pd.read_csv("ade_gpt3_fewshot_tp.csv")
nyt = pd.read_csv("nyt_gpt3_fewshot_tp.csv")
conll = pd.read_csv("conll_gpt3_fewshot_tp.csv")

In [3]:
print (ade.shape, nyt.shape, conll.shape)

(614, 4) (3731, 4) (150, 4)


In [4]:
df = pd.concat([ade, nyt, conll]).sample(frac=1)
df.head(10)

,input,relations,prefix,gold_relations
3095,"In Southold , residents can pay by the bag -- ...","[['Southold', 'LOC_CONTAINS', 'Cutchogue']]",NYT,"[['Southold', 'LOC_CONTAINS', 'Cutchogue']]"
382,replicate brain magnetic resonance imaging exa...,"[['penicillamine', 'brain lesions']]",ADE,"[['penicillamine', 'brain lesions']]"
2100,"In western New York , between Rochester and Bu...","[['New York', 'LOC_CONTAINS', 'Buffalo']]",NYT,"[['New York', 'LOC_CONTAINS', 'Buffalo'], ['Ne..."
2645,"In 2002 , it made its largest overseas investm...","[['Belgium', 'LOC_CONTAINS', 'Antwerp']]",NYT,"[['Belgium', 'LOC_CONTAINS', 'Antwerp']]"
636,This statue proves that Jesus Christ is Lord o...,"[['Tennessee', 'LOC_CONTAINS', 'Memphis']]",NYT,"[['Tennessee', 'LOC_CONTAINS', 'Memphis']]"
23,I think federalism is a very civilized way of ...,"[['Iraq', 'LOC_CONTAINS', 'Baghdad']]",NYT,"[['Iraq', 'LOC_CONTAINS', 'Baghdad'], ['Iraq',..."
345,The central orb and rods -- most are chromed a...,"[['Long Island City', 'LOC_NEIGHBORHOOD', 'Que...",NYT,"[['Long Island City', 'LOC_NEIGHBORHOOD', 'Que..."
3641,"The envoy , Terje Roed-Larsen , announced the ...","[['Syria', 'LOC_CONTAINS', 'Aleppo']]",NYT,"[['Syria', 'LOC_ADMIN_DIVISIONS', 'Aleppo'], [..."
2820,-LRB- '' Ai n't Supposed to Die a Natural Deat...,"[['New York', 'LOC_CONTAINS', 'Manhattan']]",NYT,"[['New York', 'LOC_CONTAINS', 'Manhattan']]"
785,"Suave , tan and channeling '' Miami Vice '' in...","[['Chechnya', 'LOC_COUNTRY', 'Russia'], ['Russ...",NYT,"[['Chechnya', 'LOC_COUNTRY', 'Russia'], ['Russ..."


In [5]:
train, test = train_test_split(df, test_size=0.20, stratify=df["prefix"])

In [6]:
print (train.shape, test.shape)

(3596, 4) (899, 4)


In [7]:
model_args = {
    "cache_dir" : "/scratch/wadhwa.s/cache/",
#     "device_map" : "auto",
#     "use_multiprocessing": False,
#     "use_multiprocessed_decoding": False,
    "config" : {"cache_dir" : "/scratch/wadhwa.s/cache/"},
    "max_seq_length" : 512,
    "train_batch_size" : 4,
    "eval_batch_size" :4,
    "max_length" : 200,
         "num_train_epochs" : 4,
         "no_save" : True,
#         "preprocess_inputs" : False,
         "overwrite_output_dir" : True,
#         "special_tokens_list" : ["<bos>", "<eos>", "<rel>", "<ent>"],
#         "max_length" : 200,
#         "num_beams" : 5,
         "learning_rate": 3e-5,

    }

train = train.rename(columns={"input": "input_text", "relations": "target_text"})
test = test.rename(columns={"input": "input_text", "relations": "target_text"})

train_df = train[["prefix", "input_text", "target_text"]]
test_df = test[["prefix", "input_text", "target_text"]]

In [8]:
print (train_df.shape, test_df.shape)

(3596, 3) (899, 3)


In [9]:
model = T5Model("t5", "t5-large", args=model_args)

In [10]:
model.train_model(train)

Epoch 1 of 4:   0%|                                                                   | 0/4 [00:18<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_13559/4159083419.py:1 in <cell line: 1>                                           │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_13559/4159083419.py'                        │
│                                                                                                  │
│ /home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/simpletransformers/t5/t5_model.py:22 │
│ 9 in train_model                                                                                 │
│                                                                                                  │
│    226 │   │                                                                                     │
│    227 │   │   os.makedirs(output_dir, exist_ok=True)                                            │
│    228 │   │                                                                                     │
│ ❱  229 │   │   global_step, training_details = self.train(                                       │
│    230 │   │   │   train_dataset,                                                                │
│    231 │   │   │   output_dir,                                                                   │
│    232 │   │   │   show_running_loss=show_running_loss,                                          │
│                                                                                                  │
│ /home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/simpletransformers/t5/t5_model.py:54 │
│ 8 in train                                                                                       │
│                                                                                                  │
│    545 │   │   │   │   │   loss = loss / args.gradient_accumulation_steps                        │
│    546 │   │   │   │                                                                             │
│    547 │   │   │   │   if args.fp16:                                                             │
│ ❱  548 │   │   │   │   │   scaler.scale(loss).backward()                                         │
│    549 │   │   │   │   else:                                                                     │
│    550 │   │   │   │   │   loss.backward()                                                       │
│    551                                                                                           │
│                                                                                                  │
│ /home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/torch/_tensor.py:396 in backward     │
│                                                                                                  │
│    393 │   │   │   │   retain_graph=retain_graph,                                                │
│    394 │   │   │   │   create_graph=create_graph,                                                │
│    395 │   │   │   │   inputs=inputs)                                                            │
│ ❱  396 │   │   torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=input  │
│    397 │                                                                                         │
│    398 │   def register_hook(self, hook):                                                        │
│    399 │   │   r"""Registers a backward hook.                                                    │
│                                                                                                  │
│ /home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/torch/autograd/__init__.py:173 in    │
│ backward                                                                                         │
│                                                            

In [39]:
train = train.rename(columns={"input": "input_text", "relations": "target_text"})
train["input_text"] = train.apply(lambda x: x["prefix"] + ": " + x["input_text"], axis=1)

test = test.rename(columns={"input": "input_text", "relations": "target_text"})
test["input_text"] = test.apply(lambda x: x["prefix"] + ": " + x["input_text"], axis=1)

In [40]:
train = train[["input_text", "target_text"]]
test = test[["input_text", "target_text"]]
print (train.shape, test.shape)

(3596, 2) (899, 2)


In [41]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large", cache_dir="/scratch/wadhwa.s/cache")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large", cache_dir="/scratch/wadhwa.s/cache", device_map="auto")

In [42]:
optimizer=AdamW(model.parameters(), lr=3e-5, weight_decay=0.01),

args = TrainingArguments(
    learning_rate = 3e-5,
    output_dir='/scratch/wadhwa.s/outputs/',
    num_train_epochs=4,
#    max_input_length = 512,
#    max_output_length = 200,
)

In [55]:
train_dict = train.to_dict('records')
test_dict = test.to_dict('records')

In [56]:
print (len(train_dict), len(test_dict))

3596 899


In [61]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dict,
    source_text = "input_text",
    target_text = "target_text",
    args=args,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_28316/3769855561.py:1 in <cell line: 1>                                           │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_28316/3769855561.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: __init__() got an unexpected keyword argument 'source_text'

In [60]:
trainer.train()

***** Running training *****
  Num examples = 3596
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1800
  Number of trainable parameters = 783150080
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sw7. Use `wandb login --relogin` to force relogin


The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: target_text, input_text. If target_text, input_text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_28316/4032920361.py:1 in <cell line: 1>                                           │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_28316/4032920361.py'                        │
│                                                                                                  │
│ /home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/transformers/trainer.py:1501 in      │
│ train                                                                                            │
│                                                                                                  │
│   1498 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1499 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1500 │   │   )                                                                                 │
│ ❱ 1501 │   │   return inner_training_loop(                                                       │
│   1502 │   │   │   args=args,                                                                    │
│   1503 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1504 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/transformers/trainer.py:1723 in      │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1720 │   │   │   │   self._load_rng_state(resume_from_checkpoint)                              │
│   1721 │   │   │                                                                                 │
│   1722 │   │   │   step = -1                                                                     │
│ ❱ 1723 │   │   │   for step, inputs in enumerate(epoch_iterator):                                │
│   1724 │   │   │   │                                                                             │
│   1725 │   │   │   │   # Skip past any already trained steps if resuming training                │
│   1726 │   │   │   │   if steps_trained_in_current_epoch > 0:                                    │
│                                                                                                  │
│ /home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/torch/utils/data/dataloader.py:681   │
│ in __next__                                                                                      │
│                                                                                                  │
│    678 │   │   │   if self._sampler_iter is None:                                                │
│    679 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    680 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  681 │   │   │   data = self._next_data()                                                      │
│    682 │   │   │   self._num_yielded += 1                                                        │
│    683 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    684 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/torch/utils/data/dataloader.py:721   │
│ in _next_data                                              